In [ ]:
import matplotlib.pyplot as plt
from spacepy.pycdf import CDF
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import ppigrf
from datetime import datetime

# --- Define file path ---
file_path = "LR/SW_OPER_MAGA_LR_1B_20250203T000000_20250203T235959_0606_MDR_MAG_LR.cdf"
#file_path = "SW_OPER_MAGA_HR_1B_20140305T000000_20140305T235959_0602_MDR_MAG_HR.cdf"

# --- Open the CDF file and extract data ---
with CDF(file_path) as cdf:
    latitude = cdf['Latitude'][:]
    longitude = cdf['Longitude'][:]
    B_NEC = cdf['B_NEC'][:]  # Magnetic field components (North, East, Center)
    B_N = B_NEC[:, 0]  # Northward Magnetic Field Component
    B_E = B_NEC[:, 1]  # Eastward Magnetic Field Component
    B_C = B_NEC[:, 2]  # Centerward Magnetic Field Component
    radius = cdf['Radius'][:]  # Distance from Earth's center
    Timestamp = cdf['Timestamp'][:]  # Timestamp data

# --- Compute Altitude from Radius ---
earth_radius_km = 6371  # Earth's mean radius in km
altitude = (radius / 1000) - earth_radius_km  # Convert radius from meters to km

# --- Define Region of Interest ---
lat_min, lat_max = 30, 40
lon_min, lon_max = 24, 26
# --- Filter Data within the Given Latitude & Longitude Range ---
region_indices = (latitude >= lat_min) & (latitude <= lat_max) & (longitude >= lon_min) & (longitude <= lon_max)

latitude_region = latitude[region_indices]
longitude_region = longitude[region_indices]
B_N_region = B_N[region_indices]
B_E_region = B_E[region_indices]
B_C_region = B_C[region_indices]
Altitude_region = altitude[region_indices]
Timestamp_region = Timestamp[region_indices]

# --- Compute B_N, B_E, B_C using IGRF Model ---
date = datetime(2014, 3, 5)  # Date of measurement
B_N_IGRF = []
B_E_IGRF = []
B_C_IGRF = []

for lat, lon, alt in zip(latitude_region, longitude_region, Altitude_region):
    Be, Bn, Bu = ppigrf.igrf(lon, lat, alt, date)  # IGRF expects altitude in km
    B_E_IGRF.append(Be)
    B_N_IGRF.append(Bn)
    B_C_IGRF.append(Bu)

# --- Save Data to a TXT File ---
output_file_path = "txt_files/SW_OPER_MAGA_LR_1B_20250203T000000_20250203T235959_0606_MDR_MAG_LR.txt"

with open(output_file_path, "w") as file:
    # Write header
    file.write("Timestamp,Latitude,Longitude,Altitude,B_N_SWARM,B_E_SWARM,B_C_SWARM,B_N_IGRF,B_E_IGRF,B_C_IGRF (nT)\n")
    
    # Write data rows (convert NumPy values to scalars)
    for ts, lat, lon, alt, bn_swarm, be_swarm, bc_swarm, bn_igrf, be_igrf, bc_igrf in zip(
        Timestamp_region, latitude_region, longitude_region, Altitude_region, 
        B_N_region, B_E_region, B_C_region, B_N_IGRF, B_E_IGRF, B_C_IGRF):
        
        file.write(f"{ts},{float(lat):.6f},{float(lon):.6f},{float(alt):.6f},"
                   f"{float(bn_swarm):.6f},{float(be_swarm):.6f},{float(bc_swarm):.6f},"
                   f"{float(bn_igrf):.6f},{float(be_igrf):.6f},{float(bc_igrf):.6f}\n")

print(f"Filtered data with IGRF comparison saved to {output_file_path}")




change sign of B_C_IGRF

In [ ]:
import pandas as pd
import numpy as np  # Import numpy

try:
    df = pd.read_csv('figure_3_magnetic_data_HR_with_IGRF.txt', sep=',', header=None, skiprows=1) #read without header and skip first row
    df.columns = ['Timestamp', 'Latitude', 'Longitude', 'Altitude', 'B_N_SWARM', 'B_E_SWARM', 'B_C_SWARM', 'B_N_IGRF', 'B_E_IGRF', 'B_C_IGRF (nT)']  # Set columns explicitly
    df['B_C_IGRF (nT)'] = pd.to_numeric(df['B_C_IGRF (nT)'], errors='coerce') * -1 #convert to numeric and change sign
    df.to_csv('figure_3_magnetic_data_HR_with_IGRF_modified.txt', sep=',', index=False)
    print("Sign changed and saved.")
except Exception as e:
    print(f"An error occurred: {e}")